In [1]:
from src.quantlib_rag.rag.quantlib_assistant import QuantLibQuoteAssistant
from src.quantlib_rag.rag.llm_groq import create_groq_llm
from src.quantlib_rag.graph.guarded_rag import GuardedQuantLibRAG
from src.quantlib_rag.config import CHROMA_BGE_MD, LOCAL_MISTRAL_CACHE 


In [ ]:
def create_local_quantlib_assistant() -> QuantLibQuoteAssistant:

    return QuantLibQuoteAssistant(
        db_path=CHROMA_BGE_MD,
        llm_model="mistral",
        temperature=0.0,
        k_default=5,
        cache_path=LOCAL_MISTRAL_CACHE,
        use_semantic_cache=True,
    )


def main():
    # 1) asystent lokalny (Mistral + Chroma + cache)
    assistant = create_local_quantlib_assistant()

    # 2) Groq jako LLM-as-judge
    judge_llm = create_groq_llm(
        model="llama-3.3-70b-versatile",
        temperature=0.0,
    )

    guarded = GuardedQuantLibRAG(
            assistant=assistant,
            judge_llm=judge_llm,
            k_for_verification=None,  # użyje assistant.k_default
        ) 
    #question = "How to build a FlatForward yield curve in QuantLib?"
    question = "Who is Lionel Messi"
    result = guarded.invoke(question_en=question)

    print("\n=== QUESTION ===")
    print(result["question_en"])

    print("\n=== ANSWER (guarded) ===")
    print(result["answer_en"])

    print("\n=== VERIFICATION ===")
    print(result["verification"])

    print("\n=== SOURCES ===")
    for src in result["sources"]:
        print(src)    

In [3]:
main()

c:\Users\DELL\Desktop\GitHub\quantlib-rag-engine\src\quantlib_rag\rag\quantlib_index.py:35: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceBgeEmbeddings(


[EXACT CACHE HIT] returning answer from JSON cache

=== QUESTION ===
How to build a FlatForward yield curve in QuantLib?

=== ANSWER (guarded) ===
To build a FlatForward yield curve in QuantLib, you can use the `ql.FlatForward` class. Here's an example of how to create a FlatForward yield curve with a fixed rate and day counter:

```python
import QuantLib as ql

# Set the evaluation date (e.g., June 15th, 2020)
today = ql.Date(15,6,2020)
ql.Settings.instance().evaluationDate = today

# Create a FlatForward yield curve with a fixed rate and day counter (Actual360)
flat_forward_curve = ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(0.05)), ql.Actual360())
```

In this example, the FlatForward yield curve is created with a fixed rate of 5% (0.05) and Actual/360 day counter. You can adjust the evaluation date, rate, and day counter according to your needs.

=== VERIFICATION ===
{'is_grounded': True, 'out_of_scope': False, 'faithfulness_score': 5, 'reason': 'The answer is directly base